# Introduction
We have approached the problem by trying to optimize with the help of Qiskit Pulse the CX^IZ^CX, or controled phase (ZZ) interaction with additional phase shifts on both qubits, which is the keystone of the algorithm. In case of the 4 Trotter steps, it is employed 24 times.

# First approach: Using the CR Gate
   We followed : </p>
   
   [1]S. Sheldon, E. Magesan, J. M. Chow, J. M. Gambetta
        Phys. Rev. A 93, 060302(R) (2016)</p>
           
   [2]N. Sundaresan, I. Lauer, E. Pritchett, E. Magesan, P. Jurcevic, J. M. Gambetta
        PRX Quantum 1, 020318 (2020).</p>
        
   The idea was to use the CR Gate, while trying to maximze the $ZZ$ interaction rate. Additionally, it is possible to retain the original $ZX$ interaction and performe the $CX*IZ*CX$ sequence, while also inducing $ZZ$ interaction during Cross-Resonance. This would speed up the overall $CX*IZ*CX$ sequence, leading to improved fidelity.</p>
   
   The available degrees of freedom were like in [1] and [2]: 
   > - Control amplitude
   > - Control phase 
   > - Overall signal length
   > - Rotary rate
   
   The experiments were performed on the IBMq Santiago System, as the Jakarta system remained unavailable to us for a long time. The results were unsatisfactionary, as no ZZ gate fidelity above 0.4 could be reproduced.
  

# Second approach: Hardware-efficient ZZ Gate
  following : </p>
  [3] B. K. Mitchell, R. K. Naik, A. Morvan, A. Hashim, J. M. Kreikebaum, B. Marinelli, W. Lavrijsen, K. Nowrouzi, D. I. Santiago, and I. Siddiqi Phys. Rev. Lett. 127, 200502(2021), </p>
  
  it is possible to activate the ZZ interaction, by driving both coupled, fixed-frequency qubits at a frequency between $|1> -> |2>$ transition of the higher-frequency qubit and the $|0> -> |1>$ transition of the lower frequency qubit. However, exact driving frequency, pulse amplitude, phase, pulse duration, IZ and ZI corrections have to be optimized to achieve a high fidelity gate of desired conditional phase change. </p>
  
  It's worth mentioning, that the Jakarta access was granted to us very late and a single job wainting time was about a week. At the same time, other backends don't support advanced qiskit Pulse functionalities. Therefore, despite high effort to prevent it, the later part of the code could not be tested and may contain bugs. 

Import necessary packages, load the Jakarta backend

In [1]:
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister, IBMQ, execute, transpile
from qiskit.providers.aer import QasmSimulator
from qiskit.tools.monitor import job_monitor
from qiskit.circuit import Parameter

from qiskit import assemble
from qiskit.providers.aer import PulseSimulator
from qiskit.providers.aer.pulse import PulseSystemModel

# Import state tomography modules
from qiskit.ignis.verification.tomography import state_tomography_circuits, StateTomographyFitter
from qiskit.quantum_info import state_fidelity

# suppress warnings
import warnings
warnings.filterwarnings('ignore')

from qiskit import circuit, pulse, transpile, QuantumCircuit, schedule
from qiskit.pulse import Play
from qiskit.pulse.channels import ControlChannel, DriveChannel
from qiskit.pulse.library import drag as Drag
from qiskit.pulse.library import drag
from qiskit.pulse.library import GaussianSquare
import matplotlib.pyplot as plt
from qiskit.ignis.verification.tomography import process_tomography_circuits, ProcessTomographyFitter
from qiskit.opflow import Zero, One, I, X, Y, Z

(CVXPY) Apr 15 01:36:52 PM: Encountered unexpected exception importing solver GLOP:
RuntimeError('Version of ortools (9.2.9972) is too old. Expected >= 9.3.0.')
(CVXPY) Apr 15 01:36:52 PM: Encountered unexpected exception importing solver PDLP:
RuntimeError('Version of ortools (9.2.9972) is too old. Expected >= 9.3.0.')


C:\Users\di46kon\AppData\Local\Temp/ipykernel_8536/4180362312.py:12: DeprecationWarning: The qiskit.ignis package is deprecated and has been supersceded by the qiskit-experiments project. Refer to the migration guide: https://github.com/Qiskit/qiskit-ignis#migration-guide on how to migrate to the new project.
  from qiskit.ignis.verification.tomography import state_tomography_circuits, StateTomographyFitter


In [2]:
from qiskit import IBMQ
provider = IBMQ.load_account()

In [3]:
provider = IBMQ.get_provider(hub='ibm-q-community', group='ibmquantumawards', project='open-science-22')
backend = provider.get_backend('ibmq_jakarta')

In [4]:
from qiskit.circuit.library import RXXGate, RYYGate, RZZGate, CXGate, CZGate
from qiskit.quantum_info.operators import Operator

import qiskit.quantum_info as qi

In [5]:
sim_noisy_jakarta = QasmSimulator.from_backend(provider.get_backend('ibmq_jakarta'))

Find qubit 1, 3, 5 frequencies of the Jakarta system. 

In [6]:
backend.defaults().qubit_freq_est

[5236541178.342768,
 5014454403.463387,
 5108540711.748391,
 5178143350.986743,
 5213026535.205841,
 5063267493.308405,
 5300700637.722491]

The gate requires driving below lower-frequency qubit, so in this case we choose the frequency below 5014445797.182232 for the gate on qubits (1,3) and a frequency below 5063274015.042817 for the gate on qubits (3,5)

### Optimize ZZ interaction for qubits 1 and 3:

In [8]:
freq1 = backend.properties().qubits[1][2].value*1e9 #get frequency of qubit 1 and 5
freq5 = backend.properties().qubits[5][2].value*1e9

During experiments on other backends, we found that varying phase of the Drive Signal doesn't improve the gate fidelity. Therefore, it remains to optimize: </p>
> - Drive frequency on both qubits
> - Drive amplitude
> - Signal length
> - IZ and ZI correction phase after ZZ gate 

Define the parametrized schedule which will allow for optimization of the ZZ Gate:
Firstly, we optimize the ZZ gate between qubits 1 and 3. The initial values are taken from experience with other backends.

In [ ]:
Parameters = {"omega_drives": [freq1-0.08e9],
             "drive_amplitudes": [0.1],
             'phases': [2*np.pi/3],
             "pulse_lengths": [704],
             "IZ_phases" :[0],
             'ZI_phases':[0]}

Define the parameter sweeps you want to perform.

In [ ]:
omega_drives =freq1 +np.array( [0, -0.05e9 -0.1e9, -0.2e9, -0.3e9, -0.4e9, -0.5e9, -0.6e9, -0.7e9, -0.8e9, -0.9e9, -1e9])
drive_amplitudes = np.array([0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.15, 0.2, 0.25,])  #the amplitude shouldn't be too big, as explained in [3]
pulse_lengths =[704]
phases = np.linspace(0,2*np.pi*11/12, 20)
IZ_phases = [0]
ZI_phases = [0]

sweeps = {'omega_drives' : omega_drives, 
          'drive_amplitudes': drive_amplitudes,
          'phases':phases,
          'pulse_lengths': pulse_lengths,
          'IZ_phases': IZ_phases,
          'ZI_phases':ZI_phases}

Create a universal function that returns a list of parametrized ZZ gates

In [ ]:
def build_zz_scheds(qc: int, qt: int, phases = [], omega_drives =[], drive_amplitudes = [], pulse_lengths = [], IZ_phases = [], ZI_phases = [],Hadamard = False, switch_phase = False) -> np.array:

    tomo_circs = []
    zz_gate = circuit.Gate('cx', num_qubits = 2, params = [])
    
    for omega_drive in omega_drives:
        for drive_amplitude in drive_amplitudes:
            for phase in phases:
                if switch_phase ==True:
                    phase = -1* phase
                for pulse_length in pulse_lengths:
                    for IZ_phase in IZ_phases:
                        for ZI_phase in ZI_phases:

                            with pulse.build(backend) as cr_sched:
                                pulse.set_frequency(omega_drive, DriveChannel(0))
                                pulse.set_frequency(omega_drive, DriveChannel(1))
                                pulse.play(GaussianSquare(duration=pulse_length, amp=drive_amplitude, sigma=64, width=pulse_length-256, name='CR90p_d3_u3'), DriveChannel(1), name='CR90p_d3_u3')
                                pulse.play(GaussianSquare(duration=pulse_length, amp=drive_amplitude*(np.exp(phase*1j)), sigma=64, width=pulse_length-256, name='CR90p_u3'), DriveChannel(2), name='CR90p_u3')
                                pulse.barrier(qc,qt)

                                tomo_circ = circuit.QuantumCircuit(7)  #use all qubits to avoid error
                                if Hadamard == True:
                                    tomo_circ.sc(qt)  
                                tomo_circ.append(zz_gate, [qc, qt])  #apply the zz (crz) gate 
                                tomo_circ.barrier(qc, qt)
                                tomo_circ.rz(IZ_phase, qt)          #correct phase on both qubits
                                tomo_circ.rz(ZI_phase, qc)
                                tomo_circ.add_calibration(gate=zz_gate, qubits=[qc, qt], schedule=cr_sched)
                                tomo_circs.append(tomo_circ)

    tomo_circs_transpiled = transpile(tomo_circs, backend, optimization_level = 1)            
    return schedule(tomo_circs_transpiled, backend), tomo_circs

Optimize the parameters in the given order: for each one run a sweep according to the values in sweep, retain the value resulting in the highest fidelity. The initial parameters were taken from experiments on another backend, as we didn't have access to Jakarta. Additionally, after making a rough frequency sweep with a step 100 MHz, take a refined sweep with a step 5 MHz.

In [ ]:
for sweep in sweeps:
    print("sweeping:",(sweep))
    Parameters[sweep] = sweeps[sweep]                                        # the parameters for this sweep should contain the 
                                                                             # best parameters so far and the sweep
    schedules, circuits = build_zz_scheds(qc = 1, qt = 3, **Parameters)      # create the schedules
    fidelities = []
    for sched in circuits:
        target_unitaryZZ = qi.Operator(CZGate(-np.pi/2).to_matrix())
        sched.draw()
        qpt_qcs = process_tomography_circuits(sched, (1,3))
        job1 = execute(qpt_qcs[:72:], backend, shots=8192)      # it's useful to divide the job in 2 parts, as sometimes it incurrs an error
        job2 = execute(qpt_qcs[72:], backend, shots=8192)       
        job_monitor(job1)
        job_monitor(job2)
        qpt_tomo = ProcessTomographyFitter([job1.result(), job2.result()], qpt_qcs)


        # Tomographic reconstruction
        choi_fit_lstsq = qpt_tomo.fit(method='lstsq')
        zz_fidelity = qi.average_gate_fidelity(choi_fit_lstsq, target=target_unitaryZZ)  #a better metric  for a ZZ gate fidelity 
                                                                                         #would be a measure of entangling
        fidelities.append(zz_fidelity)                                                   #save a list of fidelities for the swept parameter
        print('Average CZ gate fidelity: F = {:.5f}'.format(zz_fidelity))
    
    fidelities = np.array(fidelities)
    best = sweeps[sweep][np.argmax(fidelities)]                                     # retain the parameter resulting in the best fidelity 
    Parameters[sweep] = [best]
    if sweep == 'omega_drives':                                                  # if sweeping frequency,
        print('sweeping omega_drives more refined')                              # make an exact frequency sweep around the so far best frequency
        refined_frequency = best +np.arange(-0.05e9, 0.05e9, step = 0.005e9)        
        Parameters['omega_drives'] = best +np.arange(-0.05e9, 0.05e9, step = 0.005e9)
        schedules, circuits = build_zz_scheds(qc = 1, qt = 3, **Parameters)
        fidelities = []
        for sched in circuits:
            target_unitaryZX = qi.Operator(CXGate().to_matrix())
            target_unitaryZZ = qi.Operator(CZGate(np.pi/2).to_matrix())
            sched.draw()
            # Generate process tomography circuits and run
            qpt_qcs = process_tomography_circuits(sched, (1,3))
            job1 = execute(qpt_qcs[:72:], backend, shots=8192)      # it's useful to divide the job in 2 parts, as sometimes it incurrs an error
            job2 = execute(qpt_qcs[72:], backend, shots=8192)       
            # Extract tomography data
            job_monitor(job1)
            job_monitor(job2)
            qpt_tomo = ProcessTomographyFitter([job1.result(), job2.result()], qpt_qcs)


            # Tomographic reconstruction
            choi_fit_lstsq = qpt_tomo.fit(method='lstsq')
            zz_fidelity = qi.average_gate_fidelity(choi_fit_lstsq, target=target_unitaryZZ)  # a better metric  for a ZZ gate fidelity 
                                                                                                #could be a measure of entangling
            fidelities.append(zz_fidelity)
            print('Average CZ gate fidelity: F = {:.5f}'.format(zz_fidelity))
        Parameters['omega_drives'] = [refined_frequency[np.argmax(fidelities)]]
    
    print(Parameters)
    print('done')

### After optimizing the ZZ interaction, we have to find the pulse length that will correspond to our desired phase shift = $-\pi/2$ (for 4 Trotter steps)

The following two functions were taken from a jupyter notebook on CR gate pulse optimization:

In [ ]:
backend = provider.get_backend('ibmq_jakarta')
sim_noisy_jakarta = QasmSimulator.from_backend(provider.get_backend('ibmq_jakarta'))
backend_model= PulseSystemModel.from_backend(backend)
qubit_lo_freq = backend.defaults().qubit_freq_est

def run_pulse(sched): 
    """Runs the scheduled experiment on the simulated backend.
    
    Args:
      sched: pulse schedule to run
    """
    # assemble the qobj
    test_qobj = assemble(sched,
                        backend = backend,
                        qubit_lo_freq = qubit_lo_freq,
                        meas_level = 1, 
                        meas_return = 'avg',
                        shots = 2048)
    
    # run simulation
    myjob = backend.run(test_qobj, system_model=backend_model)
    job_monitor(myjob)
    sim_result = myjob.result()
    return sim_result.get_memory(0)

def run_ham_tomo(cr_times, cr_scheds):
    """Run Hamiltonian tomography experiment and return results.
    
    Args:
      cr_times: widths of cross resonance pulses
      cr_scheds: array of pulse schedules for Ham Tomo experiment
    """
    # expectation values of target conditioned on control
    avg_t_c = np.zeros((6, len(cr_times)), dtype = complex)

    # sanity check: expectation values of control conditioned on control
    avg_c_c = np.zeros((6, len(cr_times)), dtype = complex)

    for ii in range(len(cr_scheds)):
        result = run_pulse(cr_scheds[ii])
        avg_t_c[ii % 6, ii // 6] = 1 - 2 * result[qt]
        avg_c_c[ii % 6, ii // 6] = result[qc]
        print(ii + 1, "/",len(cr_scheds), end = "\r")
        
    return np.real(avg_t_c), np.real(avg_c_c)

In [ ]:
sweeps_pulselength = {'omega_drives' : Parameters['omega_drives'], 
          'drive_amplitudes': Parameters['drive_amplitudes'],
          'phases':Parameters['phases'],
          'pulse_lengths': np.arange(320, 1280, step =32).tolist(),
          'IZ_phases': Parameters['IZ_phases'],
          'ZI_phases': Parameters['ZI_phases']}

In [ ]:
schedules, circuits = build_zz_scheds(qc = 1, qt = 3, **sweeps_pulselength, Hadamard=True)   #make a list of schedules for increasing pulse lengths
avg_t_c, avg_c_c = run_ham_tomo(sweeps_pulselength['pulse_lengths'], circuits)


x_ground =avg_t_c[0,:]         
y_ground =avg_t_c[2,:] 
z_ground =avg_t_c[4,:]
x_excited=avg_t_c[1,:]
y_excited=avg_t_c[3,:]
z_excited=avg_t_c[5,:]

SX gate transforms a $|0>$ state to a $|0>-i|1>$ state, i.e. phase $3/2 \pi$. After the completion of $CRz(-\pi/2)$, if control was in $|0>$, target should stay in phase $3/2 \pi$; if control was in $|1>$ , target should have phase $\pi$. 

It would be beneficial to fit a sinusoid to X and Y, and retrieve the pulse length that matches exactly the desired conditional phase shift. However, under the risk of fit not converging, we choose the pulse length that is the closest to the desired conditional phase shift.

In [ ]:
d_phase = np.arctan(y_ground/x_ground) - np.arctan(y_excited/x_excited)  # get the difference of target phase for control in 0 and 1
deviations = np.abs(d_phase + np.pi/2)                                 #get the deviation between -pi/2 and the retrieved phases
best_length = sweeps_pulselength['pulse_lengths'][np.argmin(deviations)]     # choose the pulse length that minimizes the deviation
Parameters['pulse_lengths'] = [best_length]
IZ = -np.arctan(y_ground/x_ground) 
IZ = IZ.tolist()[np.argmin(deviations)]#correct the target phase
Parameters['IZ'] = [IZ]
print(Parameters)

Use the established pulse length and repeat the experiment with switched qubits : this way, you can determine the required $ZI$ correction after the $CRz$ pulse.

In [ ]:
schedules, circuits = build_zz_scheds(qc = 3, qt = 1, **Parameters, Hadamard= True, switch_phase = True)   #make a list of schedules for increasing pulse lengths

#As the qubit order is reversed, the relative driving pulse phase also needs to be reversed. Other than that, the gate is symmetrical. 

avg_t_c, avg_c_c = run_ham_tomo(Parameters['pulse_lengths'], circuits)

x_ground =avg_t_c[0,:]         
y_ground =avg_t_c[2,:] 
z_ground =avg_t_c[4,:]
x_excited=avg_t_c[1,:]
y_excited=avg_t_c[3,:]
z_excited=avg_t_c[5,:]

d_phase = np.arctan(y_ground/x_ground) - # find
ZI = -d_phase[0]
ZI = ZI.tolist()
Parameters['ZI'] = [ZI]
print(Parameters)

### Repeat optimization for a ZZ gate between qubits 3 and 5

In [ ]:
Parameters2 ={"omega_drives2": [freq5-0.15e9],
             "drive_amplitudes2": [0.1],
              'phases2': [2*np.pi/3],
             "pulse_lengths2": [704],
             "IZ_phases2" : [0],
             'ZI_phases2':[0]}

In [ ]:
omega_drives =freq5 +np.array( [0, -0.05e9 -0.1e9, -0.2e9, -0.3e9, -0.4e9, -0.5e9, -0.6e9, -0.7e9, -0.8e9, -0.9e9, -1e9])
drive_amplitudes = np.array([0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.15, 0.2, 0.25,])  #the amplitude shouldn't be too big, as explained in [3]
pulse_lengths =[704]
phases = np.linspace(0,2*np.pi*11/12, 20)
IZ_phases = [0]
ZI_phases = [0]

sweeps2 = {'omega_drives2' : omega_drives, 
          'drive_amplitudes2': drive_amplitudes,
          'phases2':phases,
          'pulse_lengths2': pulse_lengths,
          'IZ_phases2': IZ_phases,
          'ZI_phases2':ZI_phases}

In [ ]:
def build_zz_scheds2(qc: int, qt: int, phases2 = [], omega_drives2 =[], drive_amplitudes2 = [], pulse_lengths2 = [], IZ_phases2 = [], ZI_phases2 = [],Hadamard = False, switch_phase = False) -> np.array:

    tomo_circs = []
    zz_gate = circuit.Gate('cx', num_qubits = 2, params = [])
    
    for omega_drive in omega_drives2:
        for drive_amplitude in drive_amplitudes2:
            for phase in phases2:
                if switch_phase ==True:
                    phase = -1* phase
                for pulse_length in pulse_lengths2:
                    for IZ_phase in IZ_phases2:
                        for ZI_phase in ZI_phases2:

                            with pulse.build(backend) as cr_sched:
                                pulse.set_frequency(omega_drive, DriveChannel(3))
                                pulse.set_frequency(omega_drive, DriveChannel(5))
                                pulse.play(GaussianSquare(duration=pulse_length, amp=drive_amplitude, sigma=64, width=pulse_length-256, name='CR90p_d3_u3'), DriveChannel(3), name='CR90p_d3_u3')
                                pulse.play(GaussianSquare(duration=pulse_length, amp=drive_amplitude*(np.exp(phase*1j)), sigma=64, width=pulse_length-256, name='CR90p_u3'), DriveChannel(5), name='CR90p_u3')
                                pulse.barrier(qc,qt)

                                tomo_circ = circuit.QuantumCircuit(7)  #use all qubits to avoid error
                                if Hadamard == True:
                                    tomo_circ.sx(qt)  
                                tomo_circ.append(zz_gate, [qc, qt])  #apply the zz gate
                                tomo_circ.barrier(qc, qt)
                                tomo_circ.rz(IZ_phase, qt)
                                tomo_circ.rz(ZI_phase, qc)
                                tomo_circ.add_calibration(gate=zz_gate, qubits=[qc, qt], schedule=cr_sched)
                                tomo_circs.append(tomo_circ)

    tomo_circs_transpiled = transpile(tomo_circs, backend, optimization_level = 1)            
    return schedule(tomo_circs_transpiled, backend), tomo_circs

In [ ]:
for sweep in sweeps2:
    print("sweeping:",(sweep))
    Parameters2[sweep] = sweeps2[sweep]                                        # the parameters for this sweep should contain the 
                                                                             #best parameters so far and the sweep
    schedules, circuits = build_zz_scheds2(qc = 3, qt = 5, **Parameters2)      # create the schedules
    fidelities = []
    for sched in circuits:
        target_unitaryZZ = qi.Operator(CZGate(-np.pi/2).to_matrix())
        sched.draw()
        qpt_qcs = process_tomography_circuits(sched, (3,5))
        job1 = execute(qpt_qcs[:72:], backend, shots=8192)      # it's useful to divide the job in 2 parts, as sometimes it incurrs an error
        job2 = execute(qpt_qcs[72:], backend, shots=8192)       
        job_monitor(job1)
        job_monitor(job2)
        qpt_tomo = ProcessTomographyFitter([job1.result(), job2.result()], qpt_qcs)


        # Tomographic reconstruction
        choi_fit_lstsq = qpt_tomo.fit(method='lstsq')
        zz_fidelity = qi.average_gate_fidelity(choi_fit_lstsq, target=target_unitaryZZ)  #a better metric  for a ZZ gate fidelity 
                                                                                            #would be a measure of entangling
        fidelities.append(zz_fidelity)                                                   #save a list of fidelities for the swept parameter
        print('Average CZ gate fidelity: F = {:.5f}'.format(zz_fidelity))
    
    fidelities = np.array(fidelities)
    best = sweeps2[sweep][np.argmax(fidelities)]                                     # retain the parameter resulting in the best fidelity 
    Parameters2[sweep] = [best]
    if sweep == 'omega_drives2':                                        # if sweeping frequency,
        print('sweeping omega_drives2 more refined')                              #make an exact frequency sweep around the so far best frequency
        refined_frequency = best +np.arange(-0.05e9, 0.05e9, step = 0.005e9)        
        Parameters2['omega_drives2'] = best +np.arange(-0.05e9, 0.05e9, step = 0.005e9)
        schedules, circuits = build_zz_scheds2(qc = 3, qt = 5, **Parameters2)
        fidelities = []
        for sched in circuits:
            target_unitaryZZ = qi.Operator(CZGate(np.pi/2).to_matrix())
            sched.draw()
            # Generate process tomography circuits and run
            qpt_qcs = process_tomography_circuits(sched, (3,5))
            job1 = execute(qpt_qcs[:72:], backend, shots=8192)      # it's useful to divide the job in 2 parts, as sometimes it incurrs an error
            job2 = execute(qpt_qcs[72:], backend, shots=8192)       
            # Extract tomography data
            job_monitor(job1)
            job_monitor(job2)
            qpt_tomo = ProcessTomographyFitter([job1.result(), job2.result()], qpt_qcs)


            # Tomographic reconstruction
            choi_fit_lstsq = qpt_tomo.fit(method='lstsq')
            zz_fidelity = qi.average_gate_fidelity(choi_fit_lstsq, target=target_unitaryZZ)  # a better metric  for a ZZ gate fidelity 
                                                                                                #would be a measure of entangling
            fidelities.append(zz_fidelity)
            print('Average CZ gate fidelity: F = {:.5f}'.format(zz_fidelity))
        Parameters2['omega_drives2'] = [refined_frequency[np.argmax(fidelities)]]
    
    print(Parameters)
    print('done')

In [ ]:
sweeps_pulselength2 = {'omega_drives2' : Parameters2['omega_drives2'], 
          'drive_amplitudes2': Parameters2['drive_amplitudes2'],
          'phases2':Parameters2['phases2'],
          'pulse_lengths2': np.arange(320, 1280, step =32).tolist(),
          'IZ_phases2': Parameters2['IZ_phases2'],
        'ZI_phases2': Parameters2['ZI_phases2']}

In [ ]:
schedules, circuits = build_zz_scheds2(qc = 3, qt = 5, **sweeps_pulselength2, Hadamard=True)   #make a list of schedules for increasing pulse lengths
avg_t_c, avg_c_c = run_ham_tomo(sweeps_pulselength2['pulse_lengths2'], circuits)


x_ground =avg_t_c[0,:]         
y_ground =avg_t_c[2,:] 
z_ground =avg_t_c[4,:]
x_excited=avg_t_c[1,:]
y_excited=avg_t_c[3,:]
z_excited=avg_t_c[5,:]

In [ ]:
d_phase = np.arctan(y_ground/x_ground) - np.arctan(y_excited/x_excited)  # get the difference of target phase for control in 0 and 1
deviations = np.abs(d_phase + np.pi/2)                              #get the deviation between -pi/2 and the retrieved phases
best_length = sweeps_pulselength2['pulse_lengths2'][np.argmin(deviations)]     # choose the pulse length that minimizes the deviation
Parameters2['pulse_lengths2'] = [best_length]
IZ = -np.arctan(y_ground/x_ground) 
IZ = IZ.tolist()[np.argmin(deviations)]#correct the target phase
Parameters2['IZ_phases2'] = [IZ]
print(Parameters2)

In [ ]:
schedules, circuits = build_zz_scheds2(qc = 5, qt = 3, **Parameters2, Hadamard= True, switch_phase = True)   #make a list of schedules for increasing pulse lengths

#As the qubit order is reversed, the relative driving pulse phase also needs to be reversed. Other than that, the gate is symmetrical. 

avg_t_c, avg_c_c = run_ham_tomo(Parameters2['pulse_lengths2'], circuits)

x_ground =avg_t_c[0,:]         
y_ground =avg_t_c[2,:] 
z_ground =avg_t_c[4,:]
x_excited=avg_t_c[1,:]
y_excited=avg_t_c[3,:]
z_excited=avg_t_c[5,:]

d_phase = np.arctan(y_ground/x_ground)
ZI = -d_phase[0]
ZI = ZI.tolist()
Parameters2['ZI_phases2'] = [ZI]
print(Parameters2)

### Build the final Trotter step using the optimized ZZ gates parameters

The solution is based on the following idea:</p>

The sequence $CX*IRz(\pi/4)*CX$ , as in the example notebook, corresponds to $Rz(\pi/4)I*IRz(\pi/4)*ZZ(-\pi/2)$.</p>

More generally, $CX*IRz(\theta)*CX$ corresponds to $Rz(\theta)I*IRz(\theta)*ZZ(-2\theta)$.</p>

Given that we have now a single entangling gate and two fast, high-fidelity single qubit gates, we can expect a significant improvement in the overall sequence fidelity. Furthermore, the algorithm with 4 Trotter steps consists of 24 of such sequences, intertwined by single qubit operations.

In [10]:
backend = provider.get_backend('ibmq_jakarta')
sim_noisy_jakarta = QasmSimulator.from_backend(provider.get_backend('ibmq_jakarta'))

In [11]:
def build_Trotter_step( omega_drives =[], drive_amplitudes = [], phases = [], pulse_lengths = [], IZ_phases = [], ZI_phases = [],
                   omega_drives2=[], drive_amplitudes2= [], phases2 = [], pulse_lengths2= [], IZ_phases2= [], ZI_phases2 = []) -> np.array:

    tomo_circs = []
    
    zz_gate = circuit.Gate('cx', num_qubits = 2, params = [])
    zz_gate2 = circuit.Gate('cx', num_qubits =2, params = [])
    for omega_drive in omega_drives:
        for drive_amplitude in drive_amplitudes:
            for phase in phases:
                for pulse_length in pulse_lengths:
                    for IZ_phase in IZ_phases:
                        for ZI_phase in ZI_phases2:#tomo_cr_params = cr_params.copy()

                            with pulse.build(backend) as sched:


                                pulse.set_frequency(omega_drive, DriveChannel(1))
                                pulse.set_frequency(omega_drive, DriveChannel(3))
                                pulse.play(GaussianSquare(duration=pulse_length, amp=drive_amplitude, sigma=64, width=pulse_length-256, name='CR90p_d3_u3'), DriveChannel(1), name='CR90p_d3_u3')
                                pulse.play(GaussianSquare(duration=pulse_length, amp=drive_amplitude*(np.exp(phase*1j)), sigma=64, width=pulse_length-256, name='CR90p_u3'), DriveChannel(3), name='CR90p_u3')



                            for omega_drive2 in omega_drives2:
                                for drive_amplitude2 in drive_amplitudes2:
                                    for pulse_length2 in pulse_lengths2:
                                        for phase2 in phases2: 
                                            for IZ_phase2 in IZ_phases2:
                                                for ZI_phase2 in ZI_phases2:
                                                    with pulse.build(backend) as sched2:


                                                        pulse.set_frequency(omega_drive2, DriveChannel(3))
                                                        pulse.set_frequency(omega_drive2, DriveChannel(5))
                                                        pulse.play(GaussianSquare(duration=pulse_length2, amp=drive_amplitude2, sigma=64, width=pulse_length2-256, name='CR90p_d3_u3'), DriveChannel(3), name='CR90p_d3_u3')
                                                        pulse.play(GaussianSquare(duration=pulse_length2, amp=drive_amplitude2*(np.exp(phase2*1j)), sigma=64, width=pulse_length2-256, name='CR90p_u3'), DriveChannel(5), name='CR90p_u3')


                                                    tomo_circ = circuit.QuantumCircuit(7)

                                                    tomo_circ.ry(np.pi/2, [1,3])          #The trotter step is a series of ZZ gates for each basis X, Y, Z , for both qubit pairs
                                                    tomo_circ.rz(np.pi/4, [1])
                                                    tomo_circ.rz(np.pi/4, [3])                        
                                                    tomo_circ.append(zz_gate, [1, 3])     #This is the same like in the example , however CX^IZ^CX was substituted with a ZZ(pi/4) gate
                                                    tomo_circ.rz(ZI_phase, [1])
                                                    tomo_circ.rz(IZ_phase, [3])
                                                    tomo_circ.ry(-np.pi/2, [1,3])
                                                    tomo_circ.rx(np.pi/2, [1,3])
                                                    tomo_circ.rz(np.pi/4, [1])
                                                    tomo_circ.rz(np.pi/4, [3])  
                                                    tomo_circ.append(zz_gate, [1,3])
                                                    tomo_circ.rz(ZI_phase, [1])
                                                    tomo_circ.rz(IZ_phase, [3])
                                                    tomo_circ.rx(-np.pi/2, [1,3])
                                                    tomo_circ.rz(np.pi/4, [1])
                                                    tomo_circ.rz(np.pi/4, [3])  
                                                    tomo_circ.append(zz_gate, [1,3])
                                                    tomo_circ.rz(ZI_phase, [1])
                                                    tomo_circ.rz(IZ_phase, [3])

                                                    
                                                    tomo_circ.barrier(1,3,5)

                                                    tomo_circ.ry(np.pi/2, [3,5])
                                                    tomo_circ.rz(np.pi/4, [3])
                                                    tomo_circ.rz(np.pi/4, [5])  
                                                    tomo_circ.append(zz_gate2, [3,5])
                                                    tomo_circ.rz(ZI_phase2, [3])
                                                    tomo_circ.rz(IZ_phase2, [5])
                                                    tomo_circ.ry(-np.pi/2, [3,5])
                                                    tomo_circ.rx(np.pi/2, [3,5])
                                                    tomo_circ.rz(np.pi/4, [3])
                                                    tomo_circ.rz(np.pi/4, [5])  
                                                    tomo_circ.append(zz_gate2, [3,5])
                                                    tomo_circ.rz(ZI_phase2, [3])
                                                    tomo_circ.rz(IZ_phase2, [5])
                                                    tomo_circ.rx(-np.pi/2, [3,5])
                                                    tomo_circ.rz(np.pi/4, [3])
                                                    tomo_circ.rz(np.pi/4, [5])  
                                                    tomo_circ.append(zz_gate2, [3,5])
                                                    tomo_circ.rz(ZI_phase2, [3])
                                                    tomo_circ.rz(IZ_phase2, [5])

                                                    tomo_circ.add_calibration(gate=zz_gate, qubits=[1,3], schedule=sched)
                                                    tomo_circ.add_calibration(gate=zz_gate2, qubits=[3,5], schedule=sched2)
                                                    tomo_circs.append(tomo_circ)

       
                
    tomo_circs_transpiled = transpile(tomo_circs, backend, optimization_level = 1)            
    return schedule(tomo_circs_transpiled, backend), tomo_circs

In [12]:
scheds, Trot_qc = build_Trotter_step(**Parameters, **Parameters2)

In [13]:
Trot_gate = Trot_qc[0].to_instruction()

With the Trotter gate, continue as in the supplemented notebook:

In [14]:
# The final time of the state evolution
target_time = np.pi

t = Parameter('t')
# Number of trotter steps
trotter_steps = 4  ### CAN BE >= 4

# Initialize quantum circuit for 3 qubits
qr = QuantumRegister(7)
qc = QuantumCircuit(qr)

# Prepare initial state (remember we are only evolving 3 of the 7 qubits on jakarta qubits (q_5, q_3, q_1) corresponding to the state |110>)
qc.x([3,5])  # DO NOT MODIFY (|q_5,q_3,q_1> = |110>)

# Simulate time evolution under H_heis3 Hamiltonian
for _ in range(trotter_steps):
    qc.append(Trot_gate, [qr[0],qr[1],qr[2], qr[3],qr[4], qr[5],qr[6]])

# Evaluate simulation at target_time (t=pi) meaning each trotter step evolves pi/trotter_steps in time
#qc = qc.bind_parameters({t: target_time/trotter_steps})

# Generate state tomography circuits to evaluate fidelity of simulation
st_qcs = state_tomography_circuits(qc, [qr[1], qr[3], qr[5]])

# Display circuit for confirmation
# st_qcs[-1].decompose().draw()  # view decomposition of trotter gates
st_qcs[-1].draw()  # only view trotter gates

┌───────────────┐┌───────────────┐┌───────────────┐»
q15_0: ─────┤0              ├┤0              ├┤0              ├»
            │               ││               ││               │»
q15_1: ─────┤1              ├┤1              ├┤1              ├»
            │               ││               ││               │»
q15_2: ─────┤2              ├┤2              ├┤2              ├»
       ┌───┐│               ││               ││               │»
q15_3: ┤ X ├┤3 circuit-2082 ├┤3 circuit-2082 ├┤3 circuit-2082 ├»
       └───┘│               ││               ││               │»
q15_4: ─────┤4              ├┤4              ├┤4              ├»
       ┌───┐│               ││               ││               │»
q15_5: ┤ X ├┤5              ├┤5              ├┤5              ├»
       └───┘│               ││               ││               │»
q15_6: ─────┤6              ├┤6              ├┤6              ├»
            └───────────────┘└───────────────┘└───────────────┘»
 c0: 3/════════════════════════════════════════════════════════»
                                                               »
«       ┌───────────────┐ ░          
«q15_0: ┤0              ├─░──────────
«       │               │ ░ ┌─┐      
«q15_1: ┤1              ├─░─┤M├──────
«       │               │ ░ └╥┘      
«q15_2: ┤2              ├─░──╫───────
«       │               │ ░  ║ ┌─┐   
«q15_3: ┤3 circuit-2082 ├─░──╫─┤M├───
«       │               │ ░  ║ └╥┘   
«q15_4: ┤4              ├─░──╫──╫────
«       │               │ ░  ║  ║ ┌─┐
«q15_5: ┤5              ├─░──╫──╫─┤M├
«       │               │ ░  ║  ║ └╥┘
«q15_6: ┤6              ├─░──╫──╫──╫─
«       └───────────────┘ ░  ║  ║  ║ 
« c0: 3/═════════════════════╩══╩══╩═
«                            0  1  2

In [15]:
shots = 8192
reps = 8
backend = sim_noisy_jakarta
# reps = 8
# backend = jakarta

jobs = []
for _ in range(reps):
    # execute
    job = execute(st_qcs, backend, shots=shots)
    print('Job ID', job.job_id())
    jobs.append(job)

Job ID 2ae960a0-f898-4967-a259-214b9ae495ec
Job ID 7b305e44-657f-462e-85ae-2b036efe7a16
Job ID c8c53fe6-c3bb-4a20-badb-153e0abc9a0f
Job ID 731ca82f-271e-4e19-a061-973472d61d36
Job ID 55773675-25ce-4481-b125-25344a08c762
Job ID 501ab7d1-ff6f-4151-98ce-45a05429809e
Job ID 4938c68b-77c7-4774-b0e4-8c2167be958f
Job ID 372942f3-a7fd-40e7-bc4c-8bd1c6324678


In [16]:
for job in jobs:
    job_monitor(job)
    try:
        if job.error_message() is not None:
            print(job.error_message())
    except:
        pass

Job Status: job has successfully run
Job Status: job has successfully run
Job Status: job has successfully run
Job Status: job has successfully run
Job Status: job has successfully run
Job Status: job has successfully run
Job Status: job has successfully run
Job Status: job has successfully run


In [17]:
# Compute the state tomography based on the st_qcs quantum circuits and the results from those ciricuits
def state_tomo(result, st_qcs):
    # The expected final state; necessary to determine state tomography fidelity
    target_state = (One^One^Zero).to_matrix()  # DO NOT MODIFY (|q_5,q_3,q_1> = |110>)
    # Fit state tomography results
    tomo_fitter = StateTomographyFitter(result, st_qcs)
    rho_fit = tomo_fitter.fit(method='lstsq')
    # Compute fidelity
    fid = state_fidelity(rho_fit, target_state)
    return fid

# Compute tomography fidelities for each repetition
fids = []
for job in jobs:
    fid = state_tomo(job.result(), st_qcs)
    fids.append(fid)
    
print('state tomography fidelity = {:.4f} \u00B1 {:.4f}'.format(np.mean(fids), np.std(fids)))

state tomography fidelity = 0.3003 ± 0.0024


### In case the optimized schedule doesn't result in a high fidelity, you can use the parameters below (which were taken from optimization on the Santiago backend) and rerun the Trotter step. The fidelity should be around 0.3003 ± 0.0024

In [9]:
Parameters = {"omega_drives": [freq1-0.08e9],
             "drive_amplitudes": [0.05],
              'phases': [2*np.pi/3],
             "pulse_lengths": [256],
             "IZ_phases" : [0],
             'ZI_phases':[0]}
Parameters2 ={"omega_drives2": [freq1-0.08e9],
             "drive_amplitudes2": [0.05],
              'phases2': [2*np.pi/3],
             "pulse_lengths2": [256],
             "IZ_phases2" : [0],
             'ZI_phases2':[0]}